In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [14]:
employee = pd.read_csv('~/Downloads/employee.csv', 
                    parse_dates=['JOB_DATE', 'HIRE_DATE'],
                    index_col='HIRE_DATE')
employee.head()

,UNIQUE_ID,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,JOB_DATE
HIRE_DATE,,,,,,,,,
2006-06-12,0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,Active,2012-10-13
2000-07-19,1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,Active,2010-09-18
2015-02-03,2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,Active,2015-02-03
1982-02-08,3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,Active,1991-05-25
1989-06-19,4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,Active,1994-10-22


Group by gender, find average salary for each

In [15]:
employee.groupby('GENDER')['BASE_SALARY'].mean().round(-2)

GENDER
Female    52200.0
Male      57400.0
Name: BASE_SALARY, dtype: float64

Find average salary based on hire date, 10 year groups

In [16]:
employee.resample('10AS')['BASE_SALARY'].mean().round(-2)

HIRE_DATE
1958-01-01     81200.0
1968-01-01    106500.0
1978-01-01     69600.0
1988-01-01     62300.0
1998-01-01     58200.0
2008-01-01     47200.0
Freq: 10AS-JAN, Name: BASE_SALARY, dtype: float64

Combine two previous and group by gender and hire date(5 year time span)

In [24]:
sal_avg = employee.groupby('GENDER').resample('5AS')['BASE_SALARY'] \
        .mean().round(-2)
sal_avg

GENDER  HIRE_DATE 
Female  1975-01-01     37500.0
        1980-01-01     55200.0
        1985-01-01     57000.0
        1990-01-01     57800.0
        1995-01-01     57400.0
        2000-01-01     53700.0
        2005-01-01     52700.0
        2010-01-01     50400.0
        2015-01-01     38600.0
Male    1958-01-01     81200.0
        1963-01-01         NaN
        1968-01-01     78100.0
        1973-01-01    163200.0
        1978-01-01     70500.0
        1983-01-01     73400.0
        1988-01-01     66300.0
        1993-01-01     63500.0
        1998-01-01     63300.0
        2003-01-01     57700.0
        2008-01-01     51800.0
        2013-01-01     43900.0
Name: BASE_SALARY, dtype: float64

In [25]:
sal_avg.unstack('GENDER')

GENDER,Female,Male
HIRE_DATE,,
1958-01-01,NaN,81200.0
1963-01-01,NaN,NaN
1968-01-01,NaN,78100.0
1973-01-01,NaN,163200.0
1975-01-01,37500.0,NaN
1978-01-01,NaN,70500.0
1980-01-01,55200.0,NaN
1983-01-01,NaN,73400.0
1985-01-01,57000.0,NaN


Not a good comparison because first male hired 1958, first female hired in 1975

In [27]:
employee[employee['GENDER'] == 'Male'].index.min() 

Timestamp('1958-12-29 00:00:00')

In [28]:
employee[employee['GENDER'] == 'Female'].index.min()

Timestamp('1975-06-09 00:00:00')

To resolve issue, group date together with gender.

In [30]:
sal_avg2 = employee.groupby(['GENDER',
                            pd.Grouper(freq='10AS')])\
                    ['BASE_SALARY'].mean().round(-2)
sal_avg2

GENDER  HIRE_DATE 
Female  1968-01-01         NaN
        1978-01-01     57100.0
        1988-01-01     57100.0
        1998-01-01     54700.0
        2008-01-01     47300.0
Male    1958-01-01     81200.0
        1968-01-01    106500.0
        1978-01-01     72300.0
        1988-01-01     64600.0
        1998-01-01     59700.0
        2008-01-01     47200.0
Name: BASE_SALARY, dtype: float64

In [31]:
sal_final = sal_avg2.unstack('GENDER')
sal_final

GENDER,Female,Male
HIRE_DATE,,
1958-01-01,NaN,81200.0
1968-01-01,NaN,106500.0
1978-01-01,57100.0,72300.0
1988-01-01,57100.0,64600.0
1998-01-01,54700.0,59700.0
2008-01-01,47300.0,47200.0


Show beginning and end time intervals.

In [32]:
years = sal_final.index.year
years_right = years + 9
sal_final.index = years.astype(str) + '-' + years_right.astype(str)
sal_final

GENDER,Female,Male
1958-1967,NaN,81200.0
1968-1977,NaN,106500.0
1978-1987,57100.0,72300.0
1988-1997,57100.0,64600.0
1998-2007,54700.0,59700.0
2008-2017,47300.0,47200.0


Alternative way is to use cut function.

In [33]:
cuts = pd.cut(employee.index.year, bins=5, precision=0)
cuts.categories.values

array([Interval(1958.0, 1970.0, closed='right'),
       Interval(1970.0, 1981.0, closed='right'),
       Interval(1981.0, 1993.0, closed='right'),
       Interval(1993.0, 2004.0, closed='right'),
       Interval(2004.0, 2016.0, closed='right')], dtype=object)

In [34]:
employee.groupby([cuts, 'GENDER'])['BASE_SALARY'] \
        .mean().unstack('GENDER').round(-2)

GENDER,Female,Male
"(1958.0, 1970.0]",NaN,85400.0
"(1970.0, 1981.0]",54400.0,72700.0
"(1981.0, 1993.0]",55700.0,69300.0
"(1993.0, 2004.0]",56500.0,62300.0
"(2004.0, 2016.0]",49100.0,49800.0
